In [ ]:
%pip install transformers
%pip install datasets
%pip install accelerate
%pip install transformers[sentencepiece]
%pip install torch
%pip install scikit-learn
%pip install evaluate
%pip install nltk
%pip install pandas
%pip install sklearn
%pip install matplotlib

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder # Não está a ser usado
from torch.nn.utils.rnn import pad_sequence # Não está a ser usado com a sua CustomDataset
from collections import Counter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score # Adicionar estas

import matplotlib.pyplot as plt # Adicionar para mostrar o plot

In [12]:
# Load your individual dataframes
train_nvuln_df = pd.read_csv('../PHP2IL/train_com_id_il_nvuln.csv')
train_vuln_df = pd.read_csv('../PHP2IL/train_com_id_il_vuln.csv')

test_nvuln_df = pd.read_csv('../PHP2IL/test_com_id_il_nvuln.csv')
test_vuln_df = pd.read_csv('../PHP2IL/test_com_id_il_vuln.csv')

# 1. Concatenate them (stack rows on top of each other)
# Ensure to apply the initial shuffle correctly to the combined sets
train_df = pd.concat([train_nvuln_df, train_vuln_df], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
test_df = pd.concat([test_nvuln_df, test_vuln_df], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)

# Split the training data into training and validation sets
train_df, val_df = train_test_split(train_df, test_size=1/9, random_state=42, shuffle=True)

# Convert the labels to integers
train_df['vulnerable'] = train_df['vulnerable'].astype(int)
val_df['vulnerable'] = val_df['vulnerable'].astype(int)
test_df['vulnerable'] = test_df['vulnerable'].astype(int)

In [13]:
def simple_tokenizer(text):
    """
    Tokeniza o texto dividindo-o por espaços.
    """
    return text.split()

# --- Construção do Vocabulário ---
print("Construindo vocabulário a partir do conjunto de treino...")
all_train_tokens = []
for snippet in train_df['file_content_in_il']:
    all_train_tokens.extend(simple_tokenizer(snippet))

counter = Counter(all_train_tokens)
vocab_size_limit = 5000 # Definir o tamanho desejado do vocabulário
vocab = {word: idx + 1 for idx, (word, _) in enumerate(counter.most_common(vocab_size_limit))}

print(f"Vocabulário construído com {len(vocab)} tokens mais frequentes.")

# Função para converter snippets em IDs de tokens
def encode_snippet(snippet, vocab_map, max_len=100):
    """
    Converte um snippet de texto em uma sequência de IDs de tokens.
    Tokens desconhecidos ou fora do vocabulário são mapeados para 0.
    A sequência é truncada ou preenchida para max_len.
    """
    tokens = simple_tokenizer(snippet)
    token_ids = [vocab_map.get(token, 0) for token in tokens]
    
    if len(token_ids) > max_len:
        return token_ids[:max_len]
    else:
        return token_ids + [0] * (max_len - len(token_ids))

# Encodings para os conjuntos
train_encodings = [encode_snippet(snippet, vocab) for snippet in train_df['file_content_in_il']]
val_encodings = [encode_snippet(snippet, vocab) for snippet in val_df['file_content_in_il']]
test_encodings = [encode_snippet(snippet, vocab) for snippet in test_df['file_content_in_il']]

Construindo vocabulário a partir do conjunto de treino...
Vocabulário construído com 101 tokens mais frequentes.


In [14]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = {'input_ids': encodings}
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx], dtype=torch.long) for key, val in self.encodings.items()}
        # **ALTERAÇÃO AQUI:** Labels devem ser torch.long para CrossEntropyLoss
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

In [15]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, output_dim, dropout):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim) # output_dim=2, como já definido
        # **ALTERAÇÃO AQUI:** Remover self.sigmoid se quiser logits brutos
        # self.sigmoid = nn.Sigmoid() 
        
    def forward(self, input_ids, attention_mask=None):
        embedded = self.embedding(input_ids)
        lstm_out, _ = self.lstm(embedded)
        final_output = lstm_out[:, -1, :] # Pegamos o último estado oculto
        out = self.fc(final_output)
        # **ALTERAÇÃO AQUI:** Retornar 'out' diretamente (que são os logits)
        # Não aplicar sigmoid/softmax aqui, pois CrossEntropyLoss espera logits
        return out 

In [ ]:
def evaluate_model(model, data_loader, device, validation=False):
    model.eval()
    criterion = torch.nn.CrossEntropyLoss() 

    all_predictions = []
    all_labels = []
    total_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(data_loader): # Adicionado 'i' para inspeção
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids) # Estes são os LOGITS (shape: batch_size, 2)
            
            # --- VERIFICAÇÃO DE DIMENSÕES ---
            if i == 0 and validation == False: # Apenas para o primeiro batch do teste/validação principal
                print("\n--- Verificando dimensões em evaluate_model (primeiro batch) ---")
                print(f"Shape de 'outputs' do modelo: {outputs.shape}")
                print(f"Shape de 'labels': {labels.shape}")
                print(f"Tipo de dados de 'labels': {labels.dtype}")
                print(f"Valores únicos de 'labels' (no batch): {torch.unique(labels)}")
                print("----------------------------------------------------------")
            # --- FIM DA VERIFICAÇÃO ---

            loss = criterion(outputs, labels) 
            total_loss += loss.item()

            predictions = torch.argmax(outputs, dim=1) 
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # ... (restante do código de evaluate_model) ...
    accuracy = (np.array(all_predictions) == np.array(all_labels)).sum() / len(all_labels)
    precision = precision_score(all_labels, all_predictions, zero_division=0)
    recall = recall_score(all_labels, all_predictions, zero_division=0)
    f1 = f1_score(all_labels, all_predictions, zero_division=0)

    avg_loss = total_loss / len(data_loader)

    if validation:
        return avg_loss, accuracy, precision 

    print(f'Test Accuracy: {accuracy:.5f}')
    print(f'Test Precision: {precision:.5f}')
    print(f'Test Recall: {recall:.5f}')
    print(f'Test F1: {f1:.5f}')

    conf_matrix = confusion_matrix(all_labels, all_predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["Não Vulnerável", "Vulnerável"])
    disp.plot(cmap="Blues")
    plt.title("Matriz de Confusão do Conjunto de Teste")
    plt.show()
    
    return accuracy, precision, recall, f1

In [ ]:
def train_model(model, train_loader, val_loader, epochs, lr, device):
    # **ALTERAÇÃO AQUI:** Usar CrossEntropyLoss
    criterion = nn.CrossEntropyLoss() 
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for i, batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            # Labels já são long do CustomDataset
            labels = batch['labels'].to(device) 

            optimizer.zero_grad()
            outputs = model(input_ids) # Estes são os LOGITS

            if i == 0: # Apenas para o primeiro batch de treino
                print("\n--- Verificando dimensões em train_model (primeiro batch) ---")
                print(f"Shape de 'outputs' do modelo: {outputs.shape}")
                print(f"Shape de 'labels': {labels.shape}")
                print(f"Tipo de dados de 'labels': {labels.dtype}")
                print(f"Valores únicos de 'labels' (no batch): {torch.unique(labels)}")
                print("-------------------------------------------------------")
            # --- FIM DA VERIFICAÇÃO ---

            
            loss = criterion(outputs, labels) 
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        val_loss, val_acc, val_prec = evaluate_model(model, val_loader, device, validation=True)
        print(f'Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(train_loader):.4f} - Val Acc: {val_acc:.4f} - Val Prec: {val_prec:.4f}')

In [18]:
# Criar os Datasets
train_dataset = CustomDataset(train_encodings, train_df['vulnerable'].tolist())
val_dataset = CustomDataset(val_encodings, val_df['vulnerable'].tolist())
test_dataset = CustomDataset(test_encodings, test_df['vulnerable'].tolist())

# Criar os dataloaders
batch_size = 30
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset))
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))

# Criar o modelo LSTM
model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim, num_layers, output_dim, dropout)

# Treinar o modelo
train_model(model, train_loader, val_loader, epochs, lr, device)

# Avaliar o modelo
evaluate_model(model, test_loader, device)

IndexError: Target 1 is out of bounds.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Hiperparâmetros
vocab_size = len(vocab) + 1 
embedding_dim = 128
hidden_dim = 128
num_layers = 12
output_dim = 2 # Perfeito para logits de 2 classes
dropout = 0.5
lr = 0.00001
epochs = 2

In [ ]:
# Criar os Datasets
train_dataset = CustomDataset(train_encodings, train_df['vulnerable'].tolist())
val_dataset = CustomDataset(val_encodings, val_df['vulnerable'].tolist())
test_dataset = CustomDataset(test_encodings, test_df['vulnerable'].tolist())

# Criar os dataloaders
batch_size = 30
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# **ALTERAÇÃO AQUI:** Use batch_size menor para val_loader e test_loader se os datasets forem muito grandes
# (len(dataset) pode causar problemas de memória para grandes datasets)
# Se os datasets de validação e teste forem pequenos, manter len(dataset) é OK.
val_loader = DataLoader(val_dataset, batch_size=batch_size) 
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Criar o modelo LSTM
model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim, num_layers, output_dim, dropout)

# Treinar o modelo
train_model(model, train_loader, val_loader, epochs, lr, device)

# Avaliar o modelo (isto já imprime e plota a matriz de confusão)
evaluate_model(model, test_loader, device)

In [ ]:
# Função para Obter Logits por Classe (copiada e ligeiramente adaptada do que discutimos)
def get_logits_per_class(model, data_loader, device):
    model.eval()
    all_logits = []
    with torch.no_grad():
        print("Obtendo logits por classe do modelo para novas instâncias...")
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            logits_batch = model(input_ids) # Obtém os logits diretamente
            all_logits.append(logits_batch.cpu()) 
            
    final_logits_tensor = torch.cat(all_logits, dim=0)
    print(f"Total de logits obtidos: {final_logits_tensor.shape[0]} amostras, {final_logits_tensor.shape[1]} classes.")
    return final_logits_tensor

# Assumindo que você quer os logits do test_df (ou de um novo conjunto de dados)
# Se `test_loader` foi criado com `batch_size=len(test_dataset)`, ele já serve
# Caso contrário, crie um novo DataLoader para inferência se precisar de um batch_size diferente.
# Por exemplo, para garantir que todos os outputs vêm de uma vez e correspondem ao DataFrame:
inference_dataset = CustomDataset(test_encodings, test_df['vulnerable'].tolist())
inference_loader = DataLoader(inference_dataset, batch_size=len(inference_dataset), shuffle=False) # Sem shuffle para manter a ordem

# Obter os logits para o conjunto de teste (ou novas instâncias)
logits_for_new_instances = get_logits_per_class(model, inference_loader, device)

print("\n--- Logits para Novas Instâncias (por Classe) ---")
print(f"Shape: {logits_for_new_instances.shape}") # Deve ser (num_samples, 2)
print("Primeiras 10 linhas de logits:")
print(logits_for_new_instances[:10])

# Extrair os logits para cada classe
# Coluna 0: logit da classe "Não Vulnerável"
# Coluna 1: logit da classe "Vulnerável"
logit_non_vulnerable = logits_for_new_instances[:, 0].cpu().numpy()
logit_vulnerable = logits_for_new_instances[:, 1].cpu().numpy()

# Calcular probabilidades (se precisar delas para a heurística)
probabilities_per_class = torch.softmax(logits_for_new_instances, dim=1)
prob_non_vulnerable = probabilities_per_class[:, 0].cpu().numpy()
prob_vulnerable = probabilities_per_class[:, 1].cpu().numpy()

# Adicionar estes valores ao seu DataFrame de teste (ou ao DataFrame das novas instâncias)
if logits_for_new_instances.shape[0] == len(test_df):
    test_df['logit_non_vulnerable'] = logit_non_vulnerable
    test_df['logit_vulnerable'] = logit_vulnerable
    test_df['prob_non_vulnerable'] = prob_non_vulnerable
    test_df['prob_vulnerable'] = prob_vulnerable
    
    print("\nNovas colunas (logits, probabilidades) adicionadas ao test_df (primeiras 5 linhas):")
    print(test_df[['file_content_in_il', 'vulnerable', 
                   'logit_non_vulnerable', 'logit_vulnerable',
                   'prob_non_vulnerable', 'prob_vulnerable']].head())
else:
    print("Número de logits do modelo não corresponde ao número de linhas no test_df. Verifique a consistência dos dados.")

